In [1]:
import os
import pandas as pd 
import numpy as np
import folium
import matplotlib.pyplot as plt
import seaborn as sns
from haversine import haversine

#한글폰트 설정
plt.rc('font', family = 'Malgun Gothic')

## 데이터 병합

In [2]:
#데이터 병합
data = pd.DataFrame()

#버스정류장 
os.chdir("C:\\Users\\admin\\PycharmProjects\\오산시")
bus_df = pd.read_csv("전국버스정류장 위치정보.csv", encoding='cp949')
df_osan_bus = bus_df[bus_df.ADMIN_NM == '오산'].reset_index(drop = True)

data['위도'] = df_osan_bus['GPS_LATI']
data['경도'] = df_osan_bus['GPS_LONG']
data['시설명'] = '버스정류장'

#도시철도
metro_df = pd.read_csv("도시철도역사정보.csv", encoding='cp949')
osan_metro_df = metro_df[metro_df['역사도로명주소'].str.contains('오산')].reset_index(drop = True)

update_data = pd.DataFrame()
update_data['위도'] = osan_metro_df['역위도']
update_data['경도'] = osan_metro_df['역경도']
update_data['시설명'] = '지하철'

data = pd.concat([data, update_data])

#어린이집  
kid_df = pd.read_csv("어린이집도로명주소 변환.csv", encoding='cp949')
kid_df = kid_df[['소재지도로명주소', 'X', 'Y']].dropna()

update_data = pd.DataFrame()
update_data['위도'] = kid_df['Y']
update_data['경도'] = kid_df['X']
update_data['시설명'] = '어린이집'
data = pd.concat([data, update_data])

#초등학교 
eschool_df = pd.read_csv("오산시 초등학교 현황.csv")

update_data = pd.DataFrame()
update_data['위도'] = eschool_df['WGS84위도']
update_data['경도'] = eschool_df['WGS84경도']
update_data['시설명'] = '초등학교'
data = pd.concat([data, update_data])

#중학교
eschool_df = pd.read_csv("오산시 중학교 현황.csv")

update_data = pd.DataFrame()
update_data['위도'] = eschool_df['WGS84위도']
update_data['경도'] = eschool_df['WGS84경도']
update_data['시설명'] = '중학교'
data = pd.concat([data, update_data])

data = data.reset_index(drop = True)

C:\Users\admin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (5,8,11,15,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\admin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data

,위도,경도,시설명
0,37.170183,127.057650,버스정류장
1,37.162667,127.037983,버스정류장
2,37.162817,127.037700,버스정류장
3,37.161317,127.038733,버스정류장
4,37.196917,127.051783,버스정류장
...,...,...,...
699,37.157888,127.078027,중학교
700,37.167778,127.068500,중학교
701,37.139757,127.081003,중학교
702,37.139447,127.072561,중학교


## 후보지 데이터

In [28]:
select_df = pd.DataFrame({'후보지': ['세남로14번길', '수목원로','금곡로','청호로','갈곶육교 인근'],
                             '위도' : [37.196023, 37.162027, 37.173792, 37.128387, 37.132654],
                             '경도' : [127.040028, 127.051036, 127.080778, 127.083632, 127.062880],                              
                            })

In [29]:
select_df

,후보지,위도,경도
0,세남로14번길,37.196023,127.040028
1,수목원로,37.162027,127.051036
2,금곡로,37.173792,127.080778
3,청호로,37.128387,127.083632
4,갈곶육교 인근,37.132654,127.062880


## 위치선정 모델링

In [6]:
def make_distance_df(selcet_lati, selcet_long, data, range_set):
    #후보지 좌표
    selcet_point = (selcet_lati, selcet_long)
    
    lati_list = []
    long_list = [] 
    name_list = [] 
    distance_list = [] 
    
    for i in range(len(data)):
        #시설물 좌표
        facility_point = (data['위도'][i], data['경도'][i])
        dist = haversine(selcet_point, facility_point, unit='km')

        if dist <= range_set:
            lati_list.append(data['위도'][i])
            long_list.append(data['경도'][i])
            name_list.append(data['시설명'][i])
            distance_list.append(dist)

    n_df = pd.DataFrame({'위도': lati_list, '경도': long_list, '시설명': name_list, '거리':distance_list})
    
    return n_df

In [7]:
def create_map(selcet_lati, selcet_long, n_df, pilename):
    NODE_NM = n_df['시설명']
    GPS_LATI = n_df['위도']
    GPS_LONG = n_df['경도']

    #지도의 중심을 지정하기 위해 위도와 경도의 평균 구하기
    lat = GPS_LATI.mean()
    long = GPS_LONG.mean()

    #지도 띄우기
    m = folium.Map([lat, long], zoom_start = 9)

    bus_coords = []
    metro_coords = []
    kid_coords = []
    eschool_coords = []
    mschool_coords = []
    
    for i in range(len(NODE_NM)):
        if NODE_NM[i] == '버스정류장':
            x = GPS_LATI[i]
            y = GPS_LONG[i]
            bus_coords.append([x, y])

    for i in range(len(bus_coords)):
        folium.Marker(
            location = bus_coords[i],
            icon=folium.Icon(color = 'green')
        ).add_to(m)

    for i in range(len(NODE_NM)):
        if NODE_NM[i] == '지하철':
            x = GPS_LATI[i]
            y = GPS_LONG[i]
            metro_coords.append([x, y])

    for i in range(len(metro_coords)):
        folium.Marker(
            location = metro_coords[i],
            icon=folium.Icon(color = 'blue')
        ).add_to(m)

    for i in range(len(NODE_NM)):
        if NODE_NM[i] == '어린이집':
            x = GPS_LATI[i]
            y = GPS_LONG[i]
            kid_coords.append([x, y])

    for i in range(len(kid_coords)):
        folium.Marker(
            location = kid_coords[i],
            icon=folium.Icon(color = 'orange')
        ).add_to(m)

    for i in range(len(NODE_NM)):
        if NODE_NM[i] == '초등학교':
            x = GPS_LATI[i]
            y = GPS_LONG[i]
            eschool_coords.append([x, y])

    for i in range(len(eschool_coords)):
        folium.Marker(
            location = eschool_coords[i],
            icon=folium.Icon(color = 'red')
        ).add_to(m)
    
    for i in range(len(NODE_NM)):
        if NODE_NM[i] == '중학교':
            x = GPS_LATI[i]
            y = GPS_LONG[i]
            mschool_coords.append([x, y])

    for i in range(len(mschool_coords)):
        folium.Marker(
            location = mschool_coords[i],
            icon=folium.Icon(color = 'black')
        ).add_to(m)

    folium.Circle(
            location = (selcet_lati, selcet_long),
            radius = 1500,
            color = 'green',
            fill = 'crimson',
        ).add_to(m)

    m.save(pilename)

## 위치시설 분석 

In [8]:
select_df

,후보지,위도,경도
0,세남로14번길,37.196023,127.040028
1,수목원로,37.162027,127.051036
2,금곡로,37.173792,127.080778
3,청호로,37.128387,127.083632
4,갈곶육교,37.132654,127.062880


In [9]:
# 1.5km 이내 
senam_df = make_distance_df(37.196023, 127.040028, data, 1.5)
#create_map(37.196023, 127.040028, senam_df, '세남로14번길 map.html')

su_df = make_distance_df(37.162027, 127.051036, data, 1.5)
#create_map(37.162027, 127.051036, su_df, '수목원로 map.html')

geumgok_df = make_distance_df(37.173792, 127.080778, data, 1.5)
#create_map(37.173792, 127.080778, geumgok_df, '금곡로 map.html')

chungho_df = make_distance_df(37.128387, 127.083632, data, 1.5)
#create_map(37.128387, 127.083632, chungho_df, '청호로 map.html')

gal_df = make_distance_df(37.132654, 127.062880, data, 1.5)
#create_map(37.132654, 127.062880, gal_df, '갈곶육교 map.html')

In [10]:
senam_df['시설명'].value_counts()

버스정류장    45
어린이집     18
초등학교      2
지하철       2
Name: 시설명, dtype: int64

In [11]:
su_df['시설명'].value_counts()

버스정류장    67
어린이집     37
초등학교      4
중학교       3
지하철       1
Name: 시설명, dtype: int64

In [12]:
geumgok_df['시설명'].value_counts()

버스정류장    16
어린이집     14
초등학교      2
중학교       1
Name: 시설명, dtype: int64

In [13]:
chungho_df['시설명'].value_counts()

버스정류장    49
어린이집     43
초등학교      3
중학교       1
Name: 시설명, dtype: int64

In [14]:
gal_df['시설명'].value_counts()

어린이집     53
버스정류장    50
초등학교      3
중학교       1
지하철       1
Name: 시설명, dtype: int64

## 세대수 분석

In [15]:
resi_df = pd.read_csv("오산시 공동주택현황_20201005.csv", encoding='cp949')

In [16]:
def make_distance_resi_df(selcet_lati, selcet_long, data, range_set):
    #후보지 좌표
    selcet_point = (selcet_lati, selcet_long)
    
    lati_list = []
    long_list = [] 
    name_list = [] 
    distance_list = [] 
    number_list = [] 
    
    for i in range(len(data)):
        #시설물 좌표
        facility_point = (data['Y좌표'][i], data['X좌표'][i])
        dist = haversine(selcet_point, facility_point, unit='km')

        if dist <= range_set:
            lati_list.append(data['Y좌표'][i])
            long_list.append(data['X좌표'][i])
            name_list.append(data['아파트명'][i])
            number_list.append(data['세대수 '][i])
            distance_list.append(dist)

    n_df = pd.DataFrame({'위도': lati_list, '경도': long_list, '시설명': name_list, '세대수': number_list,'거리':distance_list})
    
    return n_df

In [17]:
resi_senam_df = make_distance_resi_df(37.196023, 127.040028, resi_df, 1.5)

resi_su_df = make_distance_resi_df(37.162027, 127.051036, resi_df, 1.5)

resi_geumgok_df = make_distance_resi_df(37.173792, 127.080778, resi_df, 1.5)

resi_chungho_df = make_distance_resi_df(37.128387, 127.083632, resi_df, 1.5)

resi_gal_df = make_distance_resi_df(37.132654, 127.062880, resi_df, 1.5)

In [18]:
sum(resi_senam_df['세대수'])

4381

In [19]:
sum(resi_su_df['세대수'])

7013

In [20]:
sum(resi_geumgok_df['세대수'])

4028

In [21]:
sum(resi_chungho_df['세대수'])

10439

In [22]:
sum(resi_gal_df['세대수'])

11357

## 결과

In [23]:
result_df = select_df.copy()
result_df['버스정류장'] = [45, 67, 16, 49, 53]
result_df['지하철'] = [2, 1, 0, 0, 1]
result_df['어린이집'] = [18, 37, 14, 43, 53]
result_df['초등학교'] = [2,4,2,3,3]
result_df['중학교'] = [0,3,1,1,1]
result_df['세대수'] = [4381, 7013, 4028, 10439, 11357]

In [24]:
result_df

,후보지,위도,경도,버스정류장,지하철,어린이집,초등학교,중학교,세대수
0,세남로14번길,37.196023,127.040028,45,2,18,2,0,4381
1,수목원로,37.162027,127.051036,67,1,37,4,3,7013
2,금곡로,37.173792,127.080778,16,0,14,2,1,4028
3,청호로,37.128387,127.083632,49,0,43,3,1,10439
4,갈곶육교,37.132654,127.062880,53,1,53,3,1,11357


In [26]:
socore1_list = [] 
label_n = 2
for i in range(len(result_df)):
    w1 = result_df['버스정류장'][i] / max(result_df['버스정류장'])
    w2 = result_df['지하철'][i] / max(result_df['지하철'])
    socore1_list.append((w1+w2)*100/label_n)

result_df['접근성 점수'] = socore1_list

socore2_list = [] 
label_n = 3 
for i in range(len(result_df)):
    w1 = result_df['어린이집'][i] / max(result_df['어린이집'])
    w2 = result_df['초등학교'][i] / max(result_df['초등학교'])
    w3 = result_df['중학교'][i] / max(result_df['중학교'])
    socore2_list.append((w1+ w2+ w3)*100/label_n)
    
result_df['연계성 점수'] = socore2_list

label_n = 1
socore3_list = [] 
for i in range(len(result_df)):
    w1 = result_df['세대수'][i] / max(result_df['세대수'])
    socore3_list.append(w1*100/label_n)

result_df['수요 점수'] = socore3_list

result_df['평균 점수'] = (result_df['접근성 점수'] + result_df['연계성 점수'] + result_df['수요 점수']) / 3

In [30]:
result_df

,후보지,위도,경도,버스정류장,지하철,어린이집,초등학교,중학교,세대수,접근성 점수,연계성 점수,수요 점수,평균 점수
0,세남로14번길,37.196023,127.040028,45,2,18,2,0,4381,83.582090,27.987421,38.575328,50.048280
1,수목원로,37.162027,127.051036,67,1,37,4,3,7013,75.000000,89.937107,61.750462,75.562523
2,금곡로,37.173792,127.080778,16,0,14,2,1,4028,11.940299,36.582809,35.467113,27.996740
3,청호로,37.128387,127.083632,49,0,43,3,1,10439,36.567164,63.155136,91.916879,63.879727
4,갈곶육교,37.132654,127.062880,53,1,53,3,1,11357,64.552239,69.444444,100.000000,77.998894


In [35]:
result_df[['후보지','위도','경도','접근성 점수', '연계성 점수', '수요 점수', '평균 점수']]

,후보지,위도,경도,접근성 점수,연계성 점수,수요 점수,평균 점수
0,세남로14번길,37.196023,127.040028,83.582090,27.987421,38.575328,50.048280
1,수목원로,37.162027,127.051036,75.000000,89.937107,61.750462,75.562523
2,금곡로,37.173792,127.080778,11.940299,36.582809,35.467113,27.996740
3,청호로,37.128387,127.083632,36.567164,63.155136,91.916879,63.879727
4,갈곶육교,37.132654,127.062880,64.552239,69.444444,100.000000,77.998894
